In [1]:
import os
import pandas as pd
from CRISPResso2 import CRISPRessoShared, CRISPRessoPlot
from collections import defaultdict

## Loading from report folder

In [ ]:
!ls ../CRISPResso2_tests/cli_integration_tests/CRISPResso_on_params/

In [ ]:
#! unzip ../CRISPResso2_tests/cli_integration_tests/CRISPResso_on_params/Alleles_frequency_table.zip

In [ ]:
# I think this is right
df_alleles = pd.read_csv('Alleles_frequency_table.txt', sep='\t', header=0)

In [ ]:
df_alleles.columns

In [ ]:
root = "../CRISPResso_on_Linear_chr11_sm_201_left"
root = "../CRISPResso2_tests/cli_integration_tests/CRISPResso_on_params/"
crispresso2_info = CRISPRessoShared.load_crispresso_info(root)
print(crispresso2_info['results']['ref_names'])
ref_name = crispresso2_info['results']['ref_names'][0]

In [ ]:
refs = crispresso2_info['results']['refs']

In [ ]:
refs

In [ ]:
counts_total = crispresso2_info['results']['alignment_stats']['counts_total']

We're interested in figures 9 for plotting

from CRISPRessoCORE
```
plot_9_input = {
    'reference_seq': ref_seq_around_cut,
    'df_alleles': df_to_plot,
    'fig_filename_root': fig_filename_root,
    'custom_colors': custom_config["colors"],
    'MIN_FREQUENCY': args.min_frequency_alleles_around_cut_to_plot,
    'MAX_N_ROWS': args.max_rows_alleles_around_cut_to_plot,
    'SAVE_ALSO_PNG': save_png,
    'plot_cut_point': plot_cut_point,
    'sgRNA_intervals': new_sgRNA_intervals,
    'sgRNA_names': sgRNA_names,
    'sgRNA_mismatches': sgRNA_mismatches,
    'annotate_wildtype_allele': args.annotate_wildtype_allele,
}
debug('Plotting allele distribution around cut for {0}'.format(ref_name))
plot(CRISPRessoPlot.plot_alleles_table, plot_9_input)
```

`df_to_plot` is constructed with the following lines:
```
df_alleles_around_cut=CRISPRessoShared.get_dataframe_around_cut(df_alleles.loc[df_alleles['Reference_Name'] == ref_name], cut_point, plot_half_window)
count_total = counts_total[ref_name]
if args.allele_plot_pcts_only_for_assigned_reference:
    df_alleles_around_cut['%AllReads']=df_alleles_around_cut['%Reads']
    df_alleles_around_cut['%Reads']=df_alleles_around_cut['#Reads']/count_total*100

#...

df_to_plot = df_alleles_around_cut
if not args.expand_allele_plots_by_quantification:
    df_to_plot = df_alleles_around_cut.groupby(['Aligned_Sequence', 'Reference_Sequence']).sum().reset_index().set_index('Aligned_Sequence')
    df_to_plot.sort_values(by=['#Reads', 'Aligned_Sequence', 'Reference_Sequence'], inplace=True, ascending=[False, True, True])
```

Need to figure out what the cut point is supposed to be?

In [ ]:
crispresso2_info_copy = crispresso2_info.copy()

In [ ]:
sgRNA_sequences = refs[ref_name]['sgRNA_sequences']
sgRNA_cut_points = refs[ref_name]['sgRNA_cut_points']

In [ ]:
print(sgRNA_sequences)
print(sgRNA_cut_points)

In [ ]:
crispresso2_info['running_info']['args'].plot_window_size

In [ ]:
cut_point = sgRNA_cut_points[0]
plot_half_window = max(1, crispresso2_info['running_info']['args'].plot_window_size)

In [ ]:
crispresso2_info['running_info']['args'].coding_seq

In [ ]:
df_alleles_around_cut=CRISPRessoShared.get_dataframe_around_cut(df_alleles.loc[df_alleles['Reference_Name'] == ref_name], cut_point, plot_half_window)
df_to_plot = df_alleles_around_cut


## Amino Acid coding

In [ ]:
coding_seq = crispresso2_info['running_info']['args'].coding_seq
# coding_seq = coding_seq[1::]

In [ ]:
codon_to_amino_acid = {
    'TTT': 'Phe', 'TTC': 'Phe', 'TTA': 'Leu', 'TTG': 'Leu',
    'TCT': 'Ser', 'TCC': 'Ser', 'TCA': 'Ser', 'TCG': 'Ser',
    'TAT': 'Tyr', 'TAC': 'Tyr', 'TAA': 'STOP', 'TAG': 'STOP',
    'TGT': 'Cys', 'TGC': 'Cys', 'TGA': 'STOP', 'TGG': 'Trp',
    'CTT': 'Leu', 'CTC': 'Leu', 'CTA': 'Leu', 'CTG': 'Leu',
    'CCT': 'Pro', 'CCC': 'Pro', 'CCA': 'Pro', 'CCG': 'Pro',
    'CAT': 'His', 'CAC': 'His', 'CAA': 'Gln', 'CAG': 'Gln',
    'CGT': 'Arg', 'CGC': 'Arg', 'CGA': 'Arg', 'CGG': 'Arg',
    'ATT': 'Ile', 'ATC': 'Ile', 'ATA': 'Ile', 'ATG': 'Met',
    'ACT': 'Thr', 'ACC': 'Thr', 'ACA': 'Thr', 'ACG': 'Thr',
    'AAT': 'Asn', 'AAC': 'Asn', 'AAA': 'Lys', 'AAG': 'Lys',
    'AGT': 'Ser', 'AGC': 'Ser', 'AGA': 'Arg', 'AGG': 'Arg',
    'GTT': 'Val', 'GTC': 'Val', 'GTA': 'Val', 'GTG': 'Val',
    'GCT': 'Ala', 'GCC': 'Ala', 'GCA': 'Ala', 'GCG': 'Ala',
    'GAT': 'Asp', 'GAC': 'Asp', 'GAA': 'Glu', 'GAG': 'Glu',
    'GGT': 'Gly', 'GGC': 'Gly', 'GGA': 'Gly', 'GGG': 'Gly'
}

In [ ]:
while len(coding_seq) > 2:
    codon, coding_seq = coding_seq[:3], coding_seq[3:]
    print(f"{codon} -> {codon_to_amino_acid[codon]}")

## Plotting

In [2]:
df_to_plot = pd.read_csv('df_to_plot.csv', header=0, sep='\t', index_col=0)

In [3]:
df_to_plot.head()

,Reference_Sequence,Unedited,n_deleted,n_inserted,n_mutated,#Reads,%Reads
Aligned_Sequence,,,,,,,
CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG,CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG,1,0,0,0,163,78.365385
CATGGAATCCCTTCTGCA---CCTGGATCGCTTTTCCGAG,CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG,1,0,0,0,8,3.846154
CATGGAATCCCTTCTG----ACCTGGATCGCTTTTCCGAG,CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG,1,0,0,0,2,0.961538
CATGGAATCCCTTCTGCAGC--CTGGATCGCTTTTCCGAG,CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG,1,0,0,0,2,0.961538
CATGGAATCCCTTCTGCAGCCACCTGGATCGCTTTTCCGA,CATGGAATCCCTTCTGCAGC-ACCTGGATCGCTTTTCCGA,1,0,0,0,2,0.961538


In [4]:
plot_9a_inputs = {
    'reference_seq': (reference_seq:='CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG'), 
    'df_alleles': (df_alleles:=df_to_plot), 
    'fig_filename_root': (fig_filename_root:='./scripts/figures/9a'), 
    'custom_colors': (custom_colors:={
        'Substitution': '#0000FF', 
        'Insertion': '#008000', 
        'Deletion': '#FF0000', 
        'A': '#7FC97F', 
        'T': '#BEAED4', 
        'C': '#FDC086', 
        'G': '#FFFF99', 
        'N': '#C8C8C8', 
        '-': '#1E1E1E'
    }), 
    'MIN_FREQUENCY': (MIN_FREQUENCY:=0.2), 
    'MAX_N_ROWS': (MAX_N_ROWS:=50), 
    'SAVE_ALSO_PNG': (SAVE_ALSO_PNG:=True), 
    'plot_cut_point': (plot_cut_point:=False), 
    'sgRNA_intervals': (sgRNA_intervals:=[(3, 22), (99, 119), (98, 112)]), 
    'sgRNA_names': (sgRNA_names:=['hi', 'dear', '']), 
    'sgRNA_mismatches': (sgRNA_mismatches:=[[], [0], [7]]), 
    'annotate_wildtype_allele': (annotate_wildtype_allele:='')
}

In [5]:
for idx, row in df_alleles[df_alleles['%Reads']>=MIN_FREQUENCY][:MAX_N_ROWS].iterrows():
    print(idx)
    break

CATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAG


In [6]:
(
    X, # 2d array: This is the sequence converted to ints I THINK for the cmap 
    annot, # This is the sequence of bp's 
    y_labels, # this is the percentage and tallies displayed to the right ``
    insertion_dict, # I think this is key: which aligned_seq has insertion, value: where the insertion is
    per_element_annot_kws, # dict of dicts: this is for bolding the substitutions
    is_reference,
    ref_sequence_amino_acids) = CRISPRessoPlot.prep_amino_acid_table(
        df_to_plot, 
        plot_9a_inputs['reference_seq'], 
        plot_9a_inputs['MAX_N_ROWS'], 
        plot_9a_inputs['MIN_FREQUENCY']
        )


In [7]:
CRISPRessoPlot.plot_amino_acid_table(**plot_9a_inputs)